In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer,GPT2ForSequenceClassification

In [3]:
#column_names = ["target", "id", "date", "flag", "user", "text"]

# Read the CSV file with specified column names
df = pd.read_csv("first.csv", 
                 encoding="ISO-8859-1")

In [4]:

df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
target_df = df['target'].value_counts().reset_index()
target_df

,target,count
0,0,50000
1,1,50000


In [6]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer,GPT2ForSequenceClassification
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [8]:
model = GPT2ForSequenceClassification.from_pretrained('gpt2',num_labels=1)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
text=df['text']

text = text.tolist()
encoded_texts = tokenizer(text,padding=True,truncation=True,return_tensors='pt')
input_ids = encoded_texts['input_ids']
attention_mask = encoded_texts['attention_mask']
print(input_ids,attention_mask)


tensor([[   31, 31943,  5898,  ..., 50256, 50256, 50256],
        [  271,  9247,   326,  ..., 50256, 50256, 50256],
        [   31, 27827,   488,  ..., 50256, 50256, 50256],
        ...,
        [28060, 28990, 12359,  ..., 50256, 50256, 50256],
        [   31,    66,  5908,  ..., 50256, 50256, 50256],
        [  361,   345,   836,  ..., 50256, 50256, 50256]]) tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [10]:
sentiments=df['target']
sentiments = sentiments.tolist()

In [11]:
from torch.utils.data import Dataset, DataLoader

class IMDBDataset(Dataset):
    def __init__(self,input_ids,attention_mask,labels):
        self.input_ids=input_ids
        self.attention_mask=attention_mask
        self.labels=labels
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx],self.attention_mask[idx],self.labels[idx]

In [12]:
dataset = IMDBDataset(input_ids,attention_mask,torch.tensor(sentiments,dtype=torch.float32))
dataloader = DataLoader(dataset,batch_size=128,shuffle=True)
len(dataloader)

782

In [13]:
model = torch.compile(model)

In [14]:
save_loc = '/home/arjun/Desktop/skylinez/SaveWeak1'

In [15]:
torch.cuda.empty_cache()
import torch.optim as optim
import torch
from tqdm import trange
model = model.to(device)
optimizer = optim.AdamW(model.parameters(),lr=2e-4)
criterion = torch.nn.BCELoss()
num_epochs=100
for epoch in trange(num_epochs):
    l2=0
    for batch in dataloader:
        input_ids_batch,attention_mask_batch,labels_batch = [b.to(device) for b in batch]
        optimizer.zero_grad()
        output=model(input_ids_batch,attention_mask=attention_mask_batch)
        output = torch.sigmoid(output.logits)
        loss=criterion(output.squeeze(),labels_batch)
        l1=loss
        l2=l2+l1
        loss.backward()
        optimizer.step()
    print(l2/len(dataloader))   

    if epoch%20==0:
         model.save_pretrained(f'/home/arjun/Desktop/skylinez/SaveWeak1__epoch{epoch}')
         tokenizer.save_pretrained(f'/home/arjun/Desktop/skylinez/SaveWeak1__epoch{epoch}')
      

  0%|          | 0/100 [00:00<?, ?it/s]/home/arjun/AI_ENV/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:140: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


tensor(0.3800, device='cuda:0', grad_fn=<DivBackward0>)


  2%|▏         | 2/100 [16:58<13:43:03, 503.92s/it]

tensor(0.2613, device='cuda:0', grad_fn=<DivBackward0>)


  3%|▎         | 3/100 [24:30<12:56:09, 480.10s/it]

...

tensor(0.0083, device='cuda:0', grad_fn=<DivBackward0>)


 44%|████▍     | 44/100 [5:32:46<7:00:33, 450.60s/it]

tensor(0.0084, device='cuda:0', grad_fn=<DivBackward0>)


 44%|████▍     | 44/100 [5:34:03<7:05:10, 455.54s/it]


KeyboardInterrupt: 

In [ ]:
labels_batch

In [ ]:
output.squeeze()

In [ ]:
loss=criterion(output.squeeze(),labels_batch)
loss

In [ ]:
save_loc = '/home/arjun/Desktop/skylinez/SaveWeak1_final'

In [ ]:
model.save_pretrained(save_loc)
tokenizer.save_pretrained(save_loc)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(save_loc)
model = GPT2ForSequenceClassification.from_pretrained(save_loc)